One of the most commonly asked questions by Earth Engine users is - *How do I download all images in a collection*? The Earth Engine Python API comes with a `ee.batch` module that allows you to launch batch exports and manage tasks. The recommended way to do batch exports like this is to use the Python API's `ee.batch.Export` functions and use a Python for-loop to iterate and export each image. The `ee.batch` module also gives you ability to control *Tasks* - allowing you to automate exports.

> You can also export images in a collection using Javascript API in the Code Editor but this requires you to manually start the tasks for each image. This approach is fine for small number of images. You can check out the [recommended script](https://code.earthengine.google.co.in/?scriptPath=users%2Fujavalgandhi%2FEnd-to-End-GEE%3ASupplement%2FImage_Collections%2FExporting_ImageCollections).

In [1]:
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wWj4fowOVe8U7oGgE_ww8JgqQVhyUDYBtbyzG2NrsYs&tc=FTqEHsIT3TNQ73YIqUjSjZykNanXf_BjyTs28JJ-Gg8&cc=Mx0IVRkcVEA7QZ9SwJtqaNDcXq5o4OO5Z7c1WQ-1bUo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVWUKgCrFs48ntwpBIz6y2r8KzQVB5RGZhOrGP0E0armCl-QcYqJVU

Successfully saved authorization token.


In [3]:
ee.Initialize()

#### Create a Collection

In [5]:
# geoBoundries Admin Boundaries
admin1 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM0")

# Select an Admin1 Region
region = 'Angola'
admin1Filtered = admin1.filter(ee.Filter.eq('shapeName', region))
geometry = admin1Filtered.geometry()

image = ee.Image('projects/treebeing/assets/treecover2010_00N_010E')
image1 = ee.Image('projects/treebeing/assets/treecover2010_00N_020E')
image2 = ee.Image('projects/treebeing/assets/treecover2010_10S_010E')
image3 = ee.Image('projects/treebeing/assets/treecover2010_10S_020E')
# Add tree cover for 2010
exportCol = ee.ImageCollection.fromImages([image, image1, image2, image3])

#### Export All Images

Exports are done via the ``ee.batch`` module. This module allows you to automatically start an export - making it suitable for batch exports.

In [6]:
image_ids = exportCol.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  4


In [9]:
# Export with 30m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(exportCol.filter(ee.Filter.eq('system:index', image_id)).first())
task = ee.batch.Export.image.toAsset(**{
  'image': image,
  'description': 'Image Export {}'.format(i+1),
  'assetId': 'projects/treebeing/assets/tc2010',
  'region': geometry,
  'scale': 30,
  'maxPixels': 1e13})
task.start()
print('Started Task: ', i+1)

Started Task:  4


#### Manage Running/Waiting Tasks

You can manage tasks as well. Get a list of tasks and get state information on them

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

You can cancel tasks as well

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    if task_state == 'RUNNING' or task_state == 'READY':
        task.cancel()
        print('Task {} canceled'.format(task_id))
    else:
        print('Task {} state is {}'.format(task_id, task_state))

### Exercise

The code below uses the TerraClimate data and creates an ImageCollection with 12 monthly images of maximum temperature. It also extract the geometry for Australia from the LSIB collection. Add the code to start an export task for each image in the collection for australia.

- **Hint1**: TerraClimate images have a scale of 4638.3m
- **Hint2**: You need to export the image contained in the clippedImage variable

In [ ]:
import ee

lsib = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
australia = lsib.filter(ee.Filter.eq('country_na', 'Australia'))
geometry = australia.geometry()

terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
tmax = terraclimate.select('tmmx')

def scale(image):
  return image.multiply(0.1) \
    .copyProperties(image,['system:time_start'])

tmaxScaled = tmax.map(scale)

filtered = tmaxScaled \
  .filter(ee.Filter.date('2020-01-01', '2021-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image_ids = filtered.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Replace the comments with your code.

In [ ]:
for i, image_id in enumerate(image_ids):
    exportImage = ee.Image(filtered.filter(ee.Filter.eq('system:index', image_id)).first())
    # Clip the image to the region geometry
    clippedImage = exportImage.clip(geometry)

    ## Create the export task using ee.batch.Export.image.toDrive()

    ## Start the task